In [149]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, BatchNormalization

import librosa
import math

%matplotlib inline

In [150]:
s, sr_s=librosa.load('data/train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr_x=librosa.load('data/train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)
print(S.shape)
print(X.shape)
S_abs = np.abs(S)
X_abs = np.abs(X)

(513, 2459)
(513, 2459)


In [151]:
# LEARNING_RATE = 1e-3
# BATCH_SIZE = 64

# # https://riptutorial.com/tensorflow/example/19385/basic-example
# # https://towardsdatascience.com/applied-deep-learning-part-4-convolutional-neural-networks-584bc134c1e2
# # https://towardsdatascience.com/the-4-convolutional-neural-network-models-that-can-classify-your-fashion-images-9fe7f3e5399d

# # input of length -1, and dimension 513.
# x = tf.placeholder(tf.float32, [BATCH_SIZE, 1, 513])
# y_ = tf.placeholder(tf.float32, [BATCH_SIZE, 1, 513])

# # filter with width 3, and we take 513 channels as input, and output also 100 channels.
# filters = tf.Variable(tf.random_normal([3,513,100]))

# conv1 = tf.nn.conv1d(
#     x,            
#     filters=filters,
#     padding="SAME",
#     stride=1)

# l1 = tf.layers.dense(conv1, units=200,  activation=tf.nn.relu)
# l2 = tf.layers.dense(l1, units=200,  activation=tf.nn.relu)
# y = tf.layers.dense(l2, units=513, activation=tf.nn.relu)

# loss = tf.losses.mean_pairwise_squared_error(labels=y_, predictions=y)
# # train_step = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE).minimize(loss)

# sess=tf.Session()
# tf.global_variables_initializer().run(session=sess)


                                   
# train_data = tf.data.Dataset.from_tensor_slices(tf.constant(X_abs.T[:]))
# label_data = tf.data.Dataset.from_tensor_slices(tf.constant(S_abs.T[:]))
# zip_data = tf.data.Dataset.zip((train_data, label_data))
# batch_data = zip_data.batch(BATCH_SIZE).repeat()

# # try 
# # batch_data = tf.data.Dataset.from_tensor_slices((tf.constant(X_abs.T[:]), tf.constant(S_abs.T[:])).batch(BATCH_SIZE).repeat()

# iterator = batch_data.make_one_shot_iterator()
# next_batch = iterator.get_next()

# # https://www.tensorflow.org/guide/saved_model#models
# saver = tf.train.Saver()


In [152]:
# CHECK_POINT_FILE_NAME = "/tmp/hw2_1.ckpt"
# saver.restore(sess, CHECK_POINT_FILE_NAME)

# TRAIN_STEPS = 50000
# for i in range(TRAIN_STEPS+1):    
#     batch = sess.run(next_batch)
#     _, loss_value = sess.run((train_step, loss), feed_dict={x: batch[0], y_: batch[1]})
#     if i% 2000 == 0:
#         print('Training Step:' + str(i) + '  Loss =  ' + str(loss_value))
        
# # save model
# save_path = saver.save(sess, CHECK_POINT_FILE_NAME)

In [154]:
# https://towardsdatascience.com/building-a-convolutional-neural-network-cnn-in-keras-329fbbadc5f5
# Try Keras

#create model
model = Sequential()
#add model layers
model.add(Conv1D(128, kernel_size=3, activation='relu', input_shape=(513,1)))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(300, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(513, activation='relu'))

#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])


In [ ]:
X_train = X_abs.T[:].reshape(-1, 513, 1)
y_train = S_abs.T[:]

model.fit(X_train, y_train, epochs=10)



Epoch 1/10
2459/2459 [==============================] - 22s 9ms/step - loss: 0.0483 - acc: 0.2041
Epoch 2/10
2459/2459 [==============================] - 20s 8ms/step - loss: 0.0269 - acc: 0.2924
Epoch 3/10
2459/2459 [==============================] - 21s 8ms/step - loss: 0.0254 - acc: 0.3172
Epoch 4/10
2459/2459 [==============================] - 20s 8ms/step - loss: 0.0215 - acc: 0.3310
Epoch 5/10
2459/2459 [==============================] - 20s 8ms/step - loss: 0.0208 - acc: 0.3347
Epoch 6/10
2459/2459 [==============================] - 20s 8ms/step - loss: 0.0194 - acc: 0.3562
Epoch 7/10
2459/2459 [==============================] - 20s 8ms/step - loss: 0.0184 - acc: 0.3587
Epoch 8/10
2080/2459 [========================>.....] - ETA: 3s - loss: 0.0161 - acc: 0.3476

In [143]:
def denoise_sound(input_file_name, output_file_name):
    sn, sr=librosa.load(input_file_name, sr=None)
    testX=librosa.stft(sn, n_fft=1024, hop_length=512)
    print(testX.shape)
    testX_abs = np.abs(testX)
#     S_test_abs = sess.run(y, feed_dict={x: testX_abs.T}).T
    S_test_abs = model.predict(testX_abs.T.reshape(-1, 513, 1)).T.reshape(513, -1)
#     print(testX.shape, testX_abs.shape)
    ratio = (testX / testX_abs)
    Sh = np.multiply(ratio, S_test_abs)
#     print(Sh.shape)
    librosa.output.write_wav(output_file_name, librosa.istft(Sh, hop_length=512), sr)
    

In [144]:
denoise_sound('data/test_x_01.wav', 'recover_01.wav')
denoise_sound('data/test_x_02.wav', 'recover_02.wav')

(513, 142)
(513, 380)
